[![ebac_logo-data_science.png](https://github.com/PaulodiasDeveloper/Curso_EBAC-Profissao_Cientista_de_Dados/blob/main/Cientista%20de%20Dados/ebac-course-utils/media/logo/ebac_logo-data_science.png?raw=true)](https://github.com/PaulodiasDeveloper/Curso_EBAC-Profissao_Cientista_de_Dados)

<!-- <img src="https://github.com/PaulodiasDeveloper/Curso_EBAC-Profissao_Cientista_de_Dados/blob/main/Cientista%20de%20Dados/ebac-course-utils/media/logo/ebac_logo-data_science.png" alt="ebac-logo-data-science"> -->

---

<!-- # **Profissão: Cientista de Dados** -->
### **Módulo 23** | Combinação de modelos I | Exercício 1

**Aluno:** [Paulo Roberto](https://www.linkedin.com/in/paulo-roberto/)<br>
**Data:** 20 de abril de 2025.

--- 

## Tarefa 2

#### 1. Monte um passo a passo para o algoritmo RF

**Bootstrap + Feature Selection:**

- Bootstrap: Utilize amostras aleatórias com reposição do conjunto de dados de treinamento original. Isso significa que cada amostra bootstrap pode conter repetição de dados.
- Feature Selection: Em cada amostra bootstrap, selecione aleatoriamente um subconjunto de variáveis. Para problemas de classificação, geralmente é recomendado escolher a raiz quadrada do número total de variáveis. Para problemas de regressão, um terço das variáveis é comumente usado.

**Modelagem com Árvores de Decisão:**

- Treine uma árvore de decisão de forma independente em cada amostra bootstrap com as variáveis aleatórias selecionadas no passo anterior.

**Agregação:**

- Combine os resultados de cada modelo independente (cada árvore de decisão) para obter uma previsão final.

  - Classificação: Utilize o voto majoritário, onde a classe prevista com mais frequência pelos modelos individuais é selecionada como a classe final.
  - Regressão: Calcule a média das previsões dos modelos individuais.


#### 2. Explique com suas palavras o Random Forest

O **Random Forest** é uma extensão do Bagging que oferece melhorias em desempenho e resultados. Ele é um método de combinação de modelos de Machine Learning, onde cada modelo é treinado em variações do conjunto de dados original chamadas de amostras bootstrap. Essas amostras consistem em subconjuntos aleatórios do conjunto de dados original, com possíveis repetições. No entanto, o Random Forest diferencia-se ao utilizar apenas uma quantidade determinada e aleatória de variáveis em cada modelo de árvore de decisão. Essa abordagem reduz a variância dos resultados e atenua o risco de overfitting. A previsão final é obtida pela agregação dos modelos, utilizando a média para regressão e votação para classificação.

#### 3. Qual a diferença entre Bagging e Random Forest?

A principal diferença entre **Bagging** e **Random Forest** está na seleção de variáveis:

- **Bagging**: É uma técnica geral de combinação de modelos de Machine Learning utilizando amostragem aleatória com reposição dos dados. Cada modelo é treinado em diferentes amostras bootstrap do conjunto de dados original.

- **Random Forest**: É uma variação específica do Bagging que utiliza árvores de decisão. Além da amostragem aleatória com reposição, o Random Forest seleciona aleatoriamente um subconjunto de variáveis para cada árvore de decisão. Isso aumenta a robustez do modelo e reduz a correlação entre as árvores, resultando em previsões mais precisas.

#### 4. Implementar em Python o Random Forest

In [1]:
# Import das bibliotecas:
import pandas as pd
import numpy  as np

from sklearn.datasets        import load_iris
from sklearn.datasets        import load_diabetes

from sklearn.model_selection import train_test_split

from sklearn.tree            import DecisionTreeClassifier
from sklearn.metrics         import accuracy_score

from sklearn.tree            import DecisionTreeRegressor
from sklearn.metrics         import mean_squared_error, r2_score

In [2]:
# Carregar dados
X = load_iris().data
y = load_iris().target

df = pd.DataFrame(X, columns=load_iris().feature_names)
df['target'] = y

def rf_classifier(df: pd.DataFrame, num_bootstrap_samples: int = 10, test_size: float = 0.33) -> pd.DataFrame:
    # Dividir dados em treino e teste
    df_train, df_test = train_test_split(df, test_size=test_size, random_state=42)
    
    X_test = df_test.drop(['target'], axis=1)
    y_test = df_test['target'].rename('y_test')
    
    # Dicionário para os resultados das predições de cada modelo
    y_pred_bagging = pd.DataFrame()

    for i in range(num_bootstrap_samples):
        # Bootstrap
        df_bootstrap = df_train.sample(n=len(df_train), replace=True, random_state=i)
        
        X_train = df_bootstrap.drop(['target'], axis=1)
        # Feature selection
        selected_features = np.random.choice(X_train.columns, size=round(np.sqrt(X_train.shape[1])), replace=False)
        X_train = X_train[selected_features]
        
        y_train = df_bootstrap['target']
        
        # Modelagem (base learners)
        model = DecisionTreeClassifier(random_state=i)
        model.fit(X_train, y_train)
        
        # Adicionando os resultados do modelo ao DataFrame para agregação das predições
        y_pred_bagging[i] = model.predict(X_test[selected_features])
    
    # Aggregating
    y_pred = y_pred_bagging.mode(axis=1)[0]  # Agregando o valor com maior número de aparições nas predições dos modelos
    
    # Resultados
    accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
    print(f'Accuracy score: {accuracy:.2f}')
    
    return pd.concat([y_test.reset_index(drop=True), y_pred.rename('y_pred')], axis=1)

# Executar a função
resultados = rf_classifier(df=df, num_bootstrap_samples=10, test_size=0.33)
resultados


Accuracy score: 0.94


,y_test,y_pred
0,1,1.0
1,0,0.0
2,2,2.0
3,1,1.0
4,1,1.0
5,0,0.0
6,1,1.0
7,2,2.0
8,1,2.0
9,1,1.0


In [3]:
# Carregar dados
X = load_diabetes().data
y = load_diabetes().target

df = pd.DataFrame(X, columns=load_diabetes().feature_names)
df['target'] = y

def rf_regressor(df: pd.DataFrame, num_bootstrap_samples: int = 10, test_size: float = 0.33) -> pd.DataFrame:
    # Dividir dados em treino e teste
    df_train, df_test = train_test_split(df, test_size=test_size, random_state=42)
    
    X_test = df_test.drop(['target'], axis=1)
    y_test = df_test['target'].rename('y_test')
    
    # DataFrame para os resultados das predições de cada modelo
    y_pred_bagging = pd.DataFrame()

    for i in range(num_bootstrap_samples):
        # Bootstrap
        df_bootstrap = df_train.sample(n=len(df_train), replace=True, random_state=i)
        
        X_train = df_bootstrap.drop(['target'], axis=1)
        # Feature selection
        selected_features = np.random.choice(X_train.columns, size=round(X_train.shape[1] / 3), replace=False)
        X_train = X_train[selected_features]
        
        y_train = df_bootstrap['target']

        # Modelagem (base learners)
        model = DecisionTreeRegressor(random_state=i)
        model.fit(X_train, y_train)
        
        # Adicionando os resultados do modelo ao DataFrame para agregação das predições
        y_pred_bagging[i] = model.predict(X_test[selected_features])

    # Aggregating
    y_pred = y_pred_bagging.mean(axis=1).rename('y_pred')  # Agregando as predições dos modelos baseando na média dos resultados
    
    # Resultados
    mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
    r2 = r2_score(y_true=y_test, y_pred=y_pred)
    print(f'Mean squared error: {mse:.2f}')
    print(f'Coefficient of determination: {r2:.2f}')
    
    return pd.concat([y_test.reset_index(drop=True), y_pred], axis=1)

# Executar a função
resultados = rf_regressor(df=df, num_bootstrap_samples=10, test_size=0.33)
resultados


Mean squared error: 3374.82
Coefficient of determination: 0.41


,y_test,y_pred
0,219.0,204.6
1,70.0,120.5
2,202.0,119.3
3,230.0,217.4
4,111.0,121.1
...,...,...
141,139.0,167.5
142,268.0,207.2
143,69.0,148.4
144,317.0,188.2
